
# Лабораторная работа 5

## Выполнил: Морозов Алексей, М32071

## Вариант: 9, задание: 2


In [ ]:
import numpy as np
import scipy as scp
import matplotlib as mlt
import matplotlib.pyplot as plt
import pandas as pd

import matplotlib.axes._axes as axes
import matplotlib.figure as figure


## Условие

В файле $mobile\_phones.cvs$ представлены данные о мобильных телефонах

1. Построить линейную модель, где в качестве независимых переменных выступают высота, ширина экрана (атрибуты $sc\_h$ и $sc\_w$) и емкость аккумулятора (вместе со свободным коэффициентом), зависимой – масса телефона, вычислить оценки коэффициентов модели и остаточной дисперсии, построить для них доверительные интервалы, вычислить коэффициент детерминации
2. Проверить следующие подозрения:
    * Чем больше высота экрана, тем больше масса
    * Чем больше ширина экрана, тем больше масса
    * Масса зависит и от ширины экрана, и от емкости аккумулятора


## Решение

### Задача 1
